In [ ]:
#training model
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import pandas as pd
import numpy as np

df = pd.read_csv('/content/TestData.csv')

df['Date'] = pd.to_datetime(df['Date'], format='%b-%y')
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['GOR'] = (df['WT LIQ'] - df['WT Oil']) / df['WT Oil']
df['ZP_diff_12'] = np.abs(df['Z1 BHP'] - df['Z2 BHP'])
df['ZP_diff_13'] = np.abs(df['Z1 BHP'] - df['Z3 BHP'])
df['ZP_diff_23'] = np.abs(df['Z2 BHP'] - df['Z3 BHP'])
df['BHP_avg'] = df[['Z1 BHP', 'Z2 BHP', 'Z3 BHP']].mean(axis=1)
df['Prod_norm'] = df['WT Oil'] / df['WT THP']

features = ['WT LIQ', 'WT Oil', 'WT THP', 'WT WCT', 'GOR',
            'ZP_diff_12', 'ZP_diff_13', 'ZP_diff_23', 'BHP_avg', 'Prod_norm']
X = df[features]

y, unique_classes = pd.factorize(df['Anomaly Class'])
y = pd.Series(y, index=df.index)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

eval_metric = 'mlogloss' if len(unique_classes) > 2 else 'logloss'
clf = XGBClassifier(eval_metric=eval_metric)

clf.fit(X_train, y_train)

y_pred_test = clf.predict(X_test)
y_pred_test_decoded = unique_classes[y_pred_test]

df.loc[X_test.index, 'XGB Anomaly Class'] = y_pred_test_decoded
y_test_decoded = unique_classes[y_test]

print(classification_report(y_test_decoded, y_pred_test_decoded))
print(df.head())

                       precision    recall  f1-score   support

Acid Stimulation Gain       1.00      1.00      1.00         4
          Stable Prod       1.00      1.00      1.00         1
            Transient       0.75      1.00      0.86         3
   Zonal Optimization       0.00      0.00      0.00         1
           Zonal Test       0.50      0.50      0.50         2

             accuracy                           0.82        11
            macro avg       0.65      0.70      0.67        11
         weighted avg       0.75      0.82      0.78        11

        Date   Well Name  WT LIQ  WT Oil  WT THP  WT WCT  Z1 BHP  Z2 BHP  \
0 2023-07-01  cheetah-20    9896    5799     103    4140   11700   11720   
1 2023-07-01  cheetah-20    9710    5747     103    4080    8840    8890   
2 2023-07-01  cheetah-20    9166    5548     102    3947    8440    8490   
3 2023-07-01  cheetah-20    8890    5601     101    3700    8210    8250   
4 2023-07-01  cheetah-20    6503    4729     106  

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.



In [ ]:
model_path = '/content/xgb_anomaly_model.json'
clf.save_model(model_path)
print(f"Model saved to {model_path}")

Model saved to /content/xgb_anomaly_model.json


In [ ]:
# Save unique_classes for label decoding later
import pickle
with open('/content/xgb_unique_classes.pkl', 'wb') as f:
    pickle.dump(list(unique_classes), f)
print("Class labels saved.")

Class labels saved.


In [ ]:
#test the model
from xgboost import XGBClassifier
import pandas as pd
import numpy as np
import pickle

# Load model
model_path = '/content/xgb_anomaly_model.json'
clf = XGBClassifier()
clf.load_model(model_path)

# Load class labels
with open('/content/xgb_unique_classes.pkl', 'rb') as f:
    unique_classes = pickle.load(f)

# Load new data into df (same dataframe name)
df = pd.read_csv('/content/NewTestData.csv')

# Feature engineering (same as before)
df['Date'] = pd.to_datetime(df['Date'], format='%b-%y')
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['GOR'] = (df['WT LIQ'] - df['WT Oil']) / df['WT Oil']
df['ZP_diff_12'] = np.abs(df['Z1 BHP'] - df['Z2 BHP'])
df['ZP_diff_13'] = np.abs(df['Z1 BHP'] - df['Z3 BHP'])
df['ZP_diff_23'] = np.abs(df['Z2 BHP'] - df['Z3 BHP'])
df['BHP_avg'] = df[['Z1 BHP', 'Z2 BHP', 'Z3 BHP']].mean(axis=1)
df['Prod_norm'] = df['WT Oil'] / df['WT THP']

features = ['WT LIQ', 'WT Oil', 'WT THP', 'WT WCT', 'GOR',
            'ZP_diff_12', 'ZP_diff_13', 'ZP_diff_23', 'BHP_avg', 'Prod_norm']
X = df[features]

# Predict
y_pred = clf.predict(X)

# Decode class labels
y_pred_decoded = [unique_classes[i] for i in y_pred]

# Add prediction to df
df['XGB Anomaly Class'] = y_pred_decoded

print(df.head())

In [ ]:
import plotly.figure_factory as ff
from sklearn.metrics import confusion_matrix

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_test)
labels = sorted(y_test.unique())  # Use actual class labels

fig = ff.create_annotated_heatmap(
    z=cm,
    x=labels, y=labels,
    annotation_text=cm.astype(str),
    colorscale='Blues',
    showscale=True
)
fig.update_layout(
    title="Confusion Matrix",
    xaxis_title="Predicted Label",
    yaxis_title="True Label"
)
fig.show()

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import curve_fit
import plotly.graph_objs as go

# --- Step 1: Prepare time in months ---
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df = df.sort_values('Date').reset_index(drop=True)
df['Time (months)'] = (df['Date'] - df['Date'].min()).dt.days / 30.44  # approx. months

# --- Step 2: Define hyperbolic decline function ---
def hyperbolic(t, qi, Di, b):
    return qi / np.power(1 + b * Di * t, 1 / b)

# --- Step 3: Fit curve using curve_fit ---
mask = df['WT Oil'].notna()
x_data = df.loc[mask, 'Time (months)']
y_data = df.loc[mask, 'WT Oil']

# Initial guess for parameters: qi, Di, b
initial_guess = [y_data.iloc[0], 0.2, 0.5]

try:
    popt, _ = curve_fit(hyperbolic, x_data, y_data, p0=initial_guess, bounds=([0, 0, 0.01], [np.inf, 2, 2]))
    qi_fitted, Di_fitted, b_fitted = popt
    df['Fitted WT Oil'] = hyperbolic(df['Time (months)'], *popt)
except RuntimeError:
    print("Curve fitting failed.")
    df['Fitted WT Oil'] = np.nan

# Thresholds
thresholds = {
    "bhp_tolerance_ratio": 0.05,
    "watercut_threshold": 5000,
    "oil_deviation_threshold": 0.15,
    "stabilization_deviation": 0.05,
}

# Rule-based recommendation function
def generate_recommendation_and_flag(row):
    bhps = np.array([row["Z1 BHP"], row["Z2 BHP"], row["Z3 BHP"]])
    mean_bhp = bhps.mean()
    tolerance = thresholds["bhp_tolerance_ratio"] * mean_bhp

    zone_labels = ["Z1", "Z2", "Z3"]
    zone_statuses = []

    for i, bhp in enumerate(bhps):
        if bhp < (mean_bhp - tolerance):
            zone_statuses.append((zone_labels[i], "open"))
        elif bhp > (mean_bhp + tolerance):
            zone_statuses.append((zone_labels[i], "shut"))
        else:
            zone_statuses.append((zone_labels[i], "commingled"))

    zone_summary = ", ".join([f"{zone}: {status}" for zone, status in zone_statuses])
    zone_only_status = [s for _, s in zone_statuses]

    # Default message
    message = "No clear status detected."
    anomaly_flag = 0

    if "open" in zone_only_status:
        message = "At least one zone is open – monitor production. Ignore anomalies from shut zones."
        anomaly_flag = 1
    elif all(s == "commingled" for s in zone_only_status):
        if pd.isna(row["Fitted WT Oil"]):
            message = "Commingled production – decline curve missing. Cannot evaluate anomaly."
            anomaly_flag = 1
        else:
            fitted = row["Fitted WT Oil"]
            actual = row["WT Oil"]
            deviation = (actual - fitted) / fitted if fitted else 0
            wc = row["WT WCT"]
            prev_action = row.get("prev_action", "")

            if wc > thresholds["watercut_threshold"] and deviation < -thresholds["oil_deviation_threshold"]:
                message = "High water cut and oil drop – recommend zonal optimization (e.g., close high-WCT zone)."
                anomaly_flag = 1
            elif deviation < -thresholds["oil_deviation_threshold"]:
                message = f"Transient flow suspected – significant oil drop below decline curve ({deviation:.1%}). Investigate."
                anomaly_flag = 1
            elif deviation > thresholds["oil_deviation_threshold"] and wc < thresholds["watercut_threshold"]:
                message = "Oil increase and stable water cut – possible acid stimulation or improvement. Record gains."
                anomaly_flag = 1
            elif wc < thresholds["watercut_threshold"] and "high-WCT zone" in str(prev_action):
                message = "Water cut dropped after closing high-WCT zone – possible optimization success. Monitor transient flow."
                anomaly_flag = 1
            elif abs(deviation) < thresholds["stabilization_deviation"]:
                message = "Oil rate close to decline curve – stable period post optimization."
                anomaly_flag = 0
            else:
                message = "Commingled production – no anomaly detected."
                anomaly_flag = 0
    else:
        message = "All zones shut or uncertain status – no recommendation."
        anomaly_flag = 0

    return pd.Series([anomaly_flag, f"[{zone_summary}] => {message}"])

# Apply rule-based recommendations and anomaly flag
df[['Rule Based Anomaly Flag','Rule Based Recommendation']] = df.apply(generate_recommendation_and_flag, axis=1)

# Optional: View result
df.tail(10)

,Date,Well Name,WT LIQ,WT Oil,WT THP,WT WCT,Z1 BHP,Z2 BHP,Z3 BHP,Anomaly Class,...,ZP_diff_12,ZP_diff_13,ZP_diff_23,BHP_avg,Prod_norm,XGB Anomaly Class,Time (months),Fitted WT Oil,Rule Based Anomaly Flag,Rule Based Recommendation
34,2024-11-01,cheetah-20,4482,1886,96,5792,14810,7310,7200,Acid Stimulation Gain,...,7500,7610,110,9773.333333,19.645833,NaN,16.064389,1338.445732,1,"[Z1: shut, Z2: open, Z3: open] => At least one..."
35,2024-11-01,cheetah-20,4647,1751,96,6231,15070,7290,7190,Acid Stimulation Gain,...,7780,7880,100,9850.000000,18.239583,NaN,16.064389,1338.445732,1,"[Z1: shut, Z2: open, Z3: open] => At least one..."
36,2024-11-01,cheetah-20,4763,1834,95,6150,15110,7250,7070,Acid Stimulation Gain,...,7860,8040,180,9810.000000,19.305263,Acid Stimulation Gain,16.064389,1338.445732,1,"[Z1: shut, Z2: open, Z3: open] => At least one..."
37,2024-12-01,cheetah-20,4777,1828,96,6174,15470,7240,7080,Acid Stimulation Gain,...,8230,8390,160,9930.000000,19.041667,NaN,17.049934,1301.463761,1,"[Z1: shut, Z2: open, Z3: open] => At least one..."
38,2025-01-01,cheetah-20,5077,1905,98,6247,15700,7240,7090,Acid Stimulation Gain,...,8460,8610,150,10010.000000,19.438776,Acid Stimulation Gain,18.068331,1266.296827,1,"[Z1: shut, Z2: open, Z3: open] => At least one..."
39,2025-02-01,cheetah-20,4077,1384,98,6606,15840,7550,7340,Acid Stimulation Gain,...,8290,8500,210,10243.333333,14.122449,Acid Stimulation Gain,19.086728,1233.834772,1,"[Z1: shut, Z2: open, Z3: open] => At least one..."
40,2025-02-01,cheetah-20,4085,1416,98,6534,15870,7650,7480,Acid Stimulation Gain,...,8220,8390,170,10333.333333,14.448980,NaN,19.086728,1233.834772,1,"[Z1: shut, Z2: open, Z3: open] => At least one..."
41,2025-03-01,cheetah-20,5186,1900,98,6336,15910,7310,7110,Acid Stimulation Gain,...,8600,8800,200,10110.000000,19.387755,NaN,20.006570,1206.563594,1,"[Z1: shut, Z2: open, Z3: open] => At least one..."
42,2025-04-01,cheetah-20,5830,1834,115,6855,15530,7390,7170,Acid Stimulation Gain,...,8140,8360,220,10030.000000,15.947826,NaN,21.024967,1178.383100,1,"[Z1: shut, Z2: open, Z3: open] => At least one..."
43,2025-04-01,cheetah-20,5915,1956,113,6692,15360,7560,7330,Acid Stimulation Gain,...,7800,8030,230,10083.333333,17.309735,NaN,21.024967,1178.383100,1,"[Z1: shut, Z2: open, Z3: open] => At least one..."


In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

features = ['WT LIQ', 'WT Oil', 'WT THP', 'WT WCT', 'GOR',
            'ZP_diff_12', 'ZP_diff_13', 'ZP_diff_23', 'BHP_avg', 'Prod_norm']

X = df[features]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

iso_model = IsolationForest(contamination='auto', random_state=42)
# Fit the Isolation Forest model before using decision_function or predict
iso_model.fit(X_scaled)
df['IF_score'] = iso_model.decision_function(X_scaled)
df['IF_anomaly'] = iso_model.predict(X_scaled)  # -1 = anomaly
print(df.head)

<bound method NDFrame.head of          Date   Well Name  WT LIQ  WT Oil  WT THP  WT WCT  Z1 BHP  Z2 BHP  \
0  2023-07-01  cheetah-20    9896    5799     103    4140   11700   11720   
1  2023-07-01  cheetah-20    9710    5747     103    4080    8840    8890   
2  2023-07-01  cheetah-20    9166    5548     102    3947    8440    8490   
3  2023-07-01  cheetah-20    8890    5601     101    3700    8210    8250   
4  2023-07-01  cheetah-20    6503    4729     106    2727    6420    6460   
5  2023-08-01  cheetah-20    5734    4308     103    2487    5910    5930   
6  2023-09-01  cheetah-20    5000    3240     105    3520    5620    5640   
7  2023-09-01  cheetah-20    4677    3106     107    3360    5550    5570   
8  2023-10-01  cheetah-20    4809    2992     107    3777    5550    5570   
9  2023-10-01  cheetah-20    4502    2765     107    3860    5470    5490   
10 2023-12-01  cheetah-20    3414    2090     101    3886    9620    8570   
11 2023-12-01  cheetah-20    2326    1405     

In [ ]:
from xgboost import XGBClassifier
import shap
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Create labels from IForest
y_iforest = (df['IF_anomaly'] == -1).astype(int)

# Fit XGBoost model to learn IForest-based labels
clf = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
clf.fit(X, y_iforest)

# SHAP explanation
explainer = shap.Explainer(clf)
shap_values = explainer(X)

# Visual summary (optional)
shap.summary_plot(shap_values, X, show=False)

# Save the current plot to a file
plt.savefig("/content/shap_summary.png", bbox_inches='tight', dpi=300)

# Optional: close the plot if running in loops or notebooks
plt.close()


# Add top features to 'IF anomaly signal' column
def get_top_features(shap_values_row, feature_names, top_n=2):
    abs_values = np.abs(shap_values_row.values)
    top_indices = np.argsort(abs_values)[-top_n:][::-1]
    return ", ".join([feature_names[i] for i in top_indices])

df['IF anomaly signal'] = [
    get_top_features(shap_values[i], X.columns) if y_iforest.iloc[i] == 1 else ""
    for i in range(len(df))
]

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning:

[14:33:42] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.


<ipython-input-57-1cdea8c0a21c>:19: FutureWarning:

The NumPy global RNG was seeded by calling `np.random.seed`. In a future version this function will no longer use the global RNG. Pass `rng` explicitly to opt-in to the new behaviour and silence this warning.



In [ ]:
import plotly.graph_objs as go

fig = go.Figure()

# Plot signals as lines
fig.add_trace(go.Scatter(x=df['Date'], y=df['WT Oil'], mode='lines', name='WT Oil', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Fitted WT Oil'], mode='lines', name='Fitted WT Oil', line=dict(color='red')))
fig.add_trace(go.Scatter(x=df['Date'], y=df['WT THP'], mode='lines', name='THP', line=dict(color='orange', dash='dash')))
fig.add_trace(go.Scatter(x=df['Date'], y=df['WT WCT'], mode='lines', name='Water Cut', line=dict(color='purple', dash='longdash')))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Z1 BHP'], mode='lines', name='Z1 BHP', line=dict(color='green', dash='dot')))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Z2 BHP'], mode='lines', name='Z2 BHP', line=dict(color='darkgreen', dash='dot')))
fig.add_trace(go.Scatter(x=df['Date'], y=df['Z3 BHP'], mode='lines', name='Z3 BHP', line=dict(color='lightgreen', dash='dot')))

# Plot anomalies from XGB - same shape, different colors per class
xgb_anomalies = df.copy()
xgb_anomalies['XGB Anomaly Class'] = xgb_anomalies['XGB Anomaly Class'].astype(str)

# Create a color map for classes
unique_classes = xgb_anomalies['XGB Anomaly Class'].unique()
colors_xgb = ['#e41a1c', '#377eb8', '#4daf4a', '#984ea3', '#ff7f00', '#ffff33', '#a65628', '#f781bf', '#999999']
color_map = {cls: colors_xgb[i % len(colors_xgb)] for i, cls in enumerate(unique_classes)}

# Map colors
xgb_anomalies['color'] = xgb_anomalies['XGB Anomaly Class'].map(color_map)

# Plot single trace for all XGB anomalies
fig.add_trace(go.Scatter(
    x=xgb_anomalies['Date'],
    y=xgb_anomalies['WT Oil'],
    mode='markers',
    name='XGB Anomalies',
    marker=dict(
        color=xgb_anomalies['color'],
        size=10,
        symbol='x'
    ),
    customdata=xgb_anomalies[['XGB Anomaly Class']],
    hovertemplate='%{customdata[0]}<extra></extra>'
))


# Plot Rule-Based anomalies (binary flag)
rule_anomalies = df[df['Rule Based Anomaly Flag'] == 1]
fig.add_trace(go.Scatter(
    x=rule_anomalies['Date'], y=rule_anomalies['WT Oil'],
    mode='markers', name='Rule-Based Anomaly',
    marker=dict(color='black', size=8, symbol='circle'),
    customdata=rule_anomalies[['Rule Based Recommendation']],
      hovertemplate=(
      '%{customdata[0]}<extra></extra>'
      )
))

# Plot Isolation Forest anomalies (IF_anomaly == -1)
if_anomalies = df[df['IF_anomaly'] == -1]
fig.add_trace(go.Scatter(
    x=if_anomalies['Date'], y=if_anomalies['WT Oil'],
    mode='markers', name='Isolation Forest Anomaly',
    marker=dict(color='orange', size=8, symbol='diamond'),
    customdata=if_anomalies[['IF anomaly signal']],
      hovertemplate=(
      '%{customdata[0]}<extra></extra>'
      )
))

# Layout update
fig.update_layout(
    title="Anomaly Detection using Supervised, Unsupervised and Rule based models",
    xaxis_title="Date",
    yaxis_title="Values",
    template="plotly_white",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    height=600,
)

fig.show()
fig.write_html("/content/plot-Anomaly Detection 3 modes.html")

In [ ]:
df.head()

,Date,Well Name,WT LIQ,WT Oil,WT THP,WT WCT,Z1 BHP,Z2 BHP,Z3 BHP,Anomaly Class,...,BHP_avg,Prod_norm,XGB Anomaly Class,Fitted WT Oil,Rule Based Anomaly Flag,Rule Based Recommendation,IF_score,IF_anomaly,Time (months),IF anomaly signal
0,2023-07-01,cheetah-20,9896,5799,103,4140,11700,11720,11580,Transient,...,11666.666667,56.300971,NaN,5442.11546,0,"[Z1: commingled, Z2: commingled, Z3: commingle...",-0.078005,-1,0.0,"WT LIQ, Prod_norm"
1,2023-07-01,cheetah-20,9710,5747,103,4080,8840,8890,8380,Transient,...,8703.333333,55.796117,NaN,5442.11546,0,"[Z1: commingled, Z2: commingled, Z3: commingle...",-0.045413,-1,0.0,"ZP_diff_23, Prod_norm"
2,2023-07-01,cheetah-20,9166,5548,102,3947,8440,8490,8150,Transient,...,8360.000000,54.392157,NaN,5442.11546,0,"[Z1: commingled, Z2: commingled, Z3: commingle...",-0.017170,-1,0.0,"ZP_diff_23, Prod_norm"
3,2023-07-01,cheetah-20,8890,5601,101,3700,8210,8250,7930,Transient,...,8130.000000,55.455446,Transient,5442.11546,0,"[Z1: commingled, Z2: commingled, Z3: commingle...",-0.033902,-1,0.0,"ZP_diff_23, Prod_norm"
4,2023-07-01,cheetah-20,6503,4729,106,2727,6420,6460,6070,Transient,...,6316.666667,44.613208,Transient,5442.11546,0,"[Z1: commingled, Z2: commingled, Z3: commingle...",-0.058105,-1,0.0,"ZP_diff_23, Prod_norm"
